# Rétropropagation de l'erreur dans les réseaux de neurones à décharges, GEI723, Automne 2024
Ce notebook présente comment l'algorithme de la descente du gradient peut être adapté pour la rétropropagation de l'erreur dans les réseaux de neurones à décharges avec des fonctions d'activation non dérivables.
Ce notebook est utilisé dans le cadre du cours GEI723 (Neuro-Computationnel) créé par Ismaël Balafrej, Ph.D. avec Jean Rouat, Ph.D., ing., professeur et adapté par Ahmad El Ferdaoussi, Ph.D. et Arnaud Yarga, étud. Ph.D.

Copyright (c) 2019-2024, Université de Sherbrooke, groupe de recherche NECOTIS. Tous droits réservés.  
Auteurs: Ismael Balafrej, Jean Rouat, adapté par Ahmad El Ferdaoussi et Arnaud Yarga

### Ce code n'est pas destiné à être rendu public et appartient aux auteurs.

Ce travail est adapté et inspiré des articles suivants:
1. Surrogate Gradient Learning in Spiking Neural Networks by Zenke & Ganguli (2018) https://arxiv.org/pdf/1901.09948.pdf
2. SLAYER: Spike Layer Error Reassignment in Time (2018) https://arxiv.org/pdf/1810.08646.pdf
3. Biologically inspired alternatives to backpropagation through time for learning in recurrent neural nets (2019) https://arxiv.org/pdf/1901.09049.pdf

## ATTENTION, code à compléter. La structure donnée est un exemple
Vous devez compléter la partie du code qui porte sur les équations des neurones afin de pouvoir faire fonctionner le notebook. L'exemple donné ici suppose une façon d'implémenter les équations. Il est possible d'implémenter autrement. En fait, il est possible de travailler soit en utilisant Euler (par exemple) pour intégrer numériquement les équations ou à partir des équations que vous avez trouvées en faisant vous même l'intégration préalable des équations différentielles.

## Dans cet exemple de code la gestion du potentiel et de l'intensité du neurone est placé dans la gestion des couches.
Vous pouvez décider de le faire autrement, par exemple dans la phase de propagation avant.

# Packages et imports

In [ ]:
%pip install quantities sparse

import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, model_selection, utils
import torch
import quantities as units
from sparse import COO


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Configuration

In [11]:
# Reproducibility
torch.manual_seed(0)
np.random.seed(0)

# Use the GPU unless there is none available.
# If you don't have a CUDA enabled GPU, I recommned using Google Colab,
# available at https://colab.research.google.com. Create a new notebook
# and then go to Runtime -> Change runtime type -> Hardware accelerator -> GPU
# Colab gives you access to up to 12 free continuous hours of a fairly recent GPU.
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


# Préparation des données

In [12]:
# Let's download the MNIST dataset, available at https://www.openml.org/d/554
# You can edit the argument data_home to the directory of your choice.
# The dataset will be downloaded there; the default directory is ~/scikit_learn_data/
X, y = datasets.fetch_openml('mnist_784', version=1, return_X_y=True, data_home=None, as_frame=False)
nb_of_samples, nb_of_features = X.shape
# X = 70k samples, 28*28 features; y = 70k samples, 1 label (string)

# Shuffle the dataset
X, y = utils.shuffle(X, y)

# Convert the labels (string) to integers for convenience
y = np.array(y, dtype=int)
nb_of_ouputs = np.max(y) + 1

# We'll normalize our input data in the range [0, 1[.
X = X / pow(2, 8)

# Conversion en décharges

In [13]:
# And convert the data to a spike train using TTFS encoding
dt = 1*units.ms
duration_per_image = 100*units.ms
absolute_duration = int(duration_per_image / dt)

time_of_spike = (1 - X) * absolute_duration  # The brighter the pixel, the earlier the spike
time_of_spike[X < .25] = 0  # "Remove" the spikes associated with darker pixels, which presumably carry less information

sample_id, neuron_idx = np.nonzero(time_of_spike)

# We use a sparse COO array to store the spikes for memory requirements
# You can use the spike_train variable as if it were a tensor of shape (nb_of_samples, nb_of_features, absolute_duration)
spike_train = COO((sample_id, neuron_idx, time_of_spike[sample_id, neuron_idx].astype(int)),
                  np.ones_like(sample_id), shape=(nb_of_samples, nb_of_features, absolute_duration))



# Split entrainement/test

In [14]:
# Split in train/test
nb_of_train_samples = int(nb_of_samples * 0.85)  # Keep 15% of the dataset for testing
train_indices = np.arange(nb_of_train_samples)
test_indices = np.arange(nb_of_train_samples, nb_of_samples)

# Création du réseau

In [15]:
# We create a 2 layer network (1 hidden, 1 output)
nb_hidden = 128  # Number of hidden neurons

w1 = torch.empty((nb_of_features, nb_hidden), device=device, dtype=torch.float, requires_grad=True)
torch.nn.init.normal_(w1, mean=0., std=.1)

w2 = torch.empty((nb_hidden, nb_of_ouputs), device=device, dtype=torch.float, requires_grad=True)
torch.nn.init.normal_(w2, mean=0., std=.1)


tensor([[ 0.0166, -0.0201, -0.0915,  ...,  0.1204, -0.0279,  0.1034],
        [ 0.0904, -0.0630, -0.0854,  ...,  0.1121, -0.0862, -0.1139],
        [ 0.0663, -0.0444,  0.0240,  ...,  0.0991, -0.0835,  0.0579],
        ...,
        [-0.2037,  0.0690,  0.0162,  ..., -0.0634,  0.0101, -0.0868],
        [ 0.0544, -0.1185,  0.1433,  ..., -0.0279, -0.0658,  0.1325],
        [ 0.0045,  0.0288,  0.0155,  ..., -0.0239, -0.0366, -0.0218]],
       requires_grad=True)

# Différentes fonctions d'activation et leur dérrivées

In [ ]:

def relu_forward(input):
    """Propagation avant pour ReLU"""
    out = torch.where(input > 0, torch.ones_like(input), torch.zeros_like(input))
    return out

def relu_backward(input):
    """Rétropropagation pour ReLU"""
    grad_input = torch.ones_like(input)
    grad_input[input < 0] = 0
    return grad_input

def leaky_relu_forward(input, alpha=0.01):
    """Propagation avant pour Leaky ReLU"""
    out = torch.where(input > 0, input, alpha * input)  # Leaky ReLU avec alpha
    return out

def leaky_relu_backward(input, alpha=0.01):
    """Rétropropagation pour Leaky ReLU"""
    grad_input = torch.ones_like(input)
    grad_input[input < 0] = alpha  # La dérivée du leaky ReLU est alpha pour les valeurs négatives
    return grad_input

def abs_relu_forward(input):
    """Propagation avant pour ReLU absolu"""
    out = torch.abs(input)  # ReLU absolu
    return out

def abs_relu_backward(input):
    """Rétropropagation pour ReLU absolu"""
    grad_input = torch.ones_like(input)
    grad_input[input < 0] = -1  # La dérivée du ReLU absolu est -1 pour les valeurs négatives
    return grad_input



# Classe pour la rétropropagation de l'erreur

In [ ]:


class SpikeFunction(torch.autograd.Function):
    """
    Cette classe permet de calculer la sortie d'une fonction lors de la propagation avant
    et de personnaliser la dérivée lors de la rétropropagation de l'erreur.
    """

    @staticmethod
    def forward(ctx, input, activation_type='relu', alpha=0.01):
        """
        Dans la passe avant, nous recevons un tenseur contenant l'entrée (potential-threshold).
        Nous appliquons la fonction d'activation choisie.
        """
        ctx.save_for_backward(input)
        ctx.activation_type = activation_type
        ctx.alpha = alpha
        
        # Choisir la fonction d'activation en fonction du type spécifié
        if activation_type == 'relu':
            out = relu_forward(input)
        elif activation_type == 'leaky_relu':
            out = leaky_relu_forward(input, alpha)
        elif activation_type == 'abs_relu':
            out = abs_relu_forward(input)
        else:
            raise ValueError(f"Activation type '{activation_type}' is not supported.")
        
        return out

    @staticmethod
    def backward(ctx, grad_output):
        """
        Dans la passe arrière, nous recevons un tenseur contenant le gradient de l'erreur par rapport à la sortie.
        Nous calculons le gradient de l'erreur par rapport à l'entrée en fonction de l'activation choisie.
        """
        input, = ctx.saved_tensors
        activation_type = ctx.activation_type
        alpha = ctx.alpha
        
        # Choisir la fonction d'activation en fonction du type spécifié
        if activation_type == 'relu':
            grad_input = relu_backward(input)
        elif activation_type == 'leaky_relu':
            grad_input = leaky_relu_backward(input, alpha)
        elif activation_type == 'abs_relu':
            grad_input = abs_relu_backward(input)
        else:
            raise ValueError(f"Activation type '{activation_type}' is not supported.")
        
        # Retourner le gradient de l'entrée
        return grad_output * grad_input





# """
# Cette class permet de calculer la sortie d'une fonction lors de la propagation avant et de personaliser la derivée lors de la retropropagation de l'erreur.
# Voir cet exemple pour plus de détails : https://pytorch.org/tutorials/beginner/examples_autograd/two_layer_net_custom_function.html
# """
# class SpikeFunction(torch.autograd.Function):
#     """
#     Dans la passe avant, nous recevons un tenseur contenant l'entrée (potential-threshold).
#     Nous appliquons la fonction Heaviside et renvoyons un tenseur contenant la sortie.
#     """
#     @staticmethod
#     def forward(ctx, input):
#         ctx.save_for_backward(input)
#         out = torch.zeros_like(input)
#         out[input > 0] = 1.0 # On génère une décharge quand (potential-threshold) > 0
#         return out

#     """
#     Dans la passe arrière, nous recevons un tenseur contenant le gradient de l'erreur par rapport à la sortie.
#     Nous calculons le gradient de l'erreur par rapport à l'entrée en utilisant la dérivée de la fonction ReLu.
#     """
#     @staticmethod
#     def backward(ctx, grad_output):
#         input, = ctx.saved_tensors
#         grad_relu = torch.ones_like(input) # La dérivée de la fonction ReLU
#         grad_relu[input < 0] = 0          # La dérivée de la fonction ReLU
#         return grad_output.clone()*grad_relu

In [ ]:
def run_spiking_layer(input_spike_train, layer_weights, tau_v=20*units.ms, tau_i=5*units.ms, v_threshold=1.0):
    """Here we implement a current-LIF dynamic in PyTorch"""

    # First, we multiply the input spike train by the weights of the current layer to get the current that will be added
    # We can calculate this beforehand because the weights are constant in the forward pass (no plasticity)
    input_current = torch.einsum("abc,bd->adc", (input_spike_train, layer_weights))  # Equivalent to a matrix multiplication for tensors of dim > 2 using Einstein's Notation

    recorded_spikes = []  # Array of the output spikes at each time t
    membrane_potential_at_t = torch.zeros((input_spike_train.shape[0], layer_weights.shape[-1]), device=device, dtype=torch.float)
    membrane_current_at_t = torch.zeros((input_spike_train.shape[0], layer_weights.shape[-1]), device=device, dtype=torch.float)

    for t in range(absolute_duration):  # For every timestep
        # Apply the leak
        membrane_potential_at_t = float(np.exp(-dt/tau_v))*membrane_potential_at_t # Using tau_v with euler or exact method
        membrane_current_at_t = float(np.exp(-dt/tau_i))*membrane_current_at_t # Using tau_i with euler or exact method

        # Select the input current at time t
        input_at_t = input_current[:, :, t]

        # Integrate the input current
        membrane_current_at_t += input_at_t

        # Integrate the input to the membrane potential
        membrane_potential_at_t += membrane_current_at_t

        # Apply the non-differentiable function
        #recorded_spikes_at_t = SpikeFunction.apply(membrane_potential_at_t - v_threshold)
        recorded_spikes_at_t = SpikeFunction.apply(membrane_potential_at_t - v_threshold, activation_type='relu')
        #activation_type='leaky_relu', alpha=0.01
        #activation_type='abs_relu'

        
        recorded_spikes.append(recorded_spikes_at_t)

        # Reset the spiked neurons
        membrane_potential_at_t[membrane_potential_at_t > v_threshold] = 0

    recorded_spikes = torch.stack(recorded_spikes, dim=2) # Stack over time axis (Array -> Tensor)
    return recorded_spikes


# Entrainement

In [ ]:
# Set-up training
nb_of_epochs = 20
batch_size = 256  # The backpropagation is done after every batch, but a batch here is also used for memory requirements
number_of_batches = len(train_indices) // batch_size

params = [w1, w2]  # Trainable parameters
optimizer = torch.optim.Adam(params, lr=0.01, amsgrad=True)
loss_fn = torch.nn.MSELoss(reduction='mean')

for e in range(nb_of_epochs):
    epoch_loss = 0
    for batch in np.array_split(train_indices, number_of_batches):
        # Select batch and convert to tensors
        batch_spike_train = torch.FloatTensor(spike_train[batch].todense()).to(device)
        batch_labels = torch.LongTensor(y[batch, np.newaxis]).to(device)

        # Here we create a target spike count (10 spikes for wrong label, 100 spikes for true label) in a one-hot fashion
        # This approach is seen in Shrestha & Orchard (2018) https://arxiv.org/pdf/1810.08646.pdf
        # Code available at https://github.com/bamsumit/slayerPytorch
        min_spike_count = 10 * torch.ones((batch.shape[0], 10), device=device, dtype=torch.float)
        target_output = min_spike_count.scatter_(1, batch_labels, 100.0)

        # Forward propagation
        layer_1_spikes = run_spiking_layer(batch_spike_train, w1)
        layer_2_spikes = run_spiking_layer(layer_1_spikes, w2)
        network_output = torch.sum(layer_2_spikes, 2)  # Count the spikes over time axis
        loss = loss_fn(network_output, target_output)

        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print("Epoch %i -- loss : %.4f" %(e+1, epoch_loss / number_of_batches))

Epoch 1 -- loss : 327.6027
Epoch 2 -- loss : 291.1524
Epoch 3 -- loss : 285.9956
Epoch 4 -- loss : 284.2993


# Test

In [ ]:
# Test the accuracy of the model
correct_label_count = 0

# We only need to batchify the test set for memory requirements
for batch in np.array_split(test_indices,  len(test_indices) // batch_size):
    test_spike_train = torch.FloatTensor(spike_train[batch].todense()).to(device)

    # Same forward propagation as before
    layer_1_spikes = run_spiking_layer(test_spike_train, w1)
    layer_2_spikes = run_spiking_layer(layer_1_spikes, w2)
    network_output = torch.sum(layer_2_spikes, 2)  # Count the spikes over time axis

    # Do the prediction by selecting the output neuron with the most number of spikes
    _, am = torch.max(network_output, 1)
    correct_label_count += np.sum(am.detach().cpu().numpy() == y[batch])

print("Model accuracy on test set: %.3f" % (correct_label_count / len(test_indices)))

# Validation